## Processing flight volume data
The input files are daily flight information about flight origin and destination by airport. This script load in the data, map airport to country, and aggregate flight volume to the country-month level. It can either aggregate by origin or desination country.
- Data comes from https://zenodo.org/records/7923702
    - The monthly file could have records not in that month. For example, a Jan 2021 file can have records for 2020, reason unknown.
- `airports.csv` comes from [https://github.com/mborsetti/airportsdata](https://github.com/mborsetti/airportsdata/blob/main/airportsdata/airports.csv)
- `countries.csv` comes from [radcliff/wikipedia-iso-country-codes.csv](https://gist.github.com/radcliff/f09c0f88344a7fcef373#file-wikipedia-iso-country-codes-csv)


In [50]:
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, TimestampType
from pyspark.sql.window import Window
from pyspark.sql import DataFrame

import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

spark = SparkSession.builder.appName("COVID-19 and Flight Volume Analysis").getOrCreate()

In [51]:
# load csv.gz file in ../data/flight_volume_raw
airports = spark.read.csv('../data/airports.csv', header=True, inferSchema=True)
countries = spark.read.csv('../data/countries.csv', header=True, inferSchema=True) \
    .withColumnRenamed('English short name lower case', 'country') \
    .withColumnRenamed('Alpha-2 code', 'country_code') \
    .select('country', 'country_code')
def load_flight_data(data_path: str, batch_date: str):
    batch_date = batch_date[:6]   # only use year and month
    df_flight = spark.read.csv(f'../data/flight_volume_raw/flightlist_{batch_date}*_{batch_date}*.csv.gz', header=True, inferSchema=True).select('day', 'origin', 'destination')
    return df_flight

df_flight = load_flight_data('../data/flight_volume_raw/flightlist_20190101_20190131.csv.gz', '201901')

df_flight.show(5)
airports.show(5)

+-------------------+------+-----------+
|                day|origin|destination|
+-------------------+------+-----------+
|2018-12-31 19:00:00|  YMML|       LFPG|
|2018-12-31 19:00:00|  YMML|       LEBL|
|2018-12-31 19:00:00|  YSSY|       EDDF|
|2018-12-31 19:00:00|  LEMD|       LEMD|
|2018-12-31 19:00:00|  YSSY|       LFPG|
+-------------------+------+-----------+
only showing top 5 rows

+----+----+--------------------+------------+-------+-------+---------+--------+----------+-----------------+----+
|icao|iata|                name|        city|   subd|country|elevation|     lat|       lon|               tz| lid|
+----+----+--------------------+------------+-------+-------+---------+--------+----------+-----------------+----+
|00AA|NULL|Aero B Ranch Airport|       Leoti| Kansas|     US|   3435.0|38.70402|-101.47391|  America/Chicago|00AA|
|00AK|NULL|        Lowell Field|Anchor Point| Alaska|     US|    252.0|59.94889|-151.69222|America/Anchorage|00AK|
|00AL|NULL|        Epps Airpark

In [52]:
# map origin and destination to country
def map_flight_data(df_flight: DataFrame, airports: DataFrame, countries: DataFrame) -> DataFrame:
    """
    Map the flight data to country and icao

    Args:
        df_flight: DataFrame, the flight data
        airports: DataFrame, the airports data

    Returns:
        df_flight_mapped: DataFrame, the flight data with country and icao
    """
    airports = airports.select('icao', 'country')
    df_flight_mapped = df_flight.join(airports, df_flight['origin'] == airports['icao'], 'left')\
        .withColumnRenamed('country', 'origin_country').withColumnRenamed('icao', 'origin_icao') \
    .join(airports, df_flight['destination'] == airports['icao'], 'left') \
    .withColumnRenamed('country', 'destination_country').withColumnRenamed('icao', 'destination_icao') \
    .select('day', 'origin_country', 'destination_country', 'origin_icao', 'destination_icao')
    df_flight_mapped = df_flight_mapped.withColumn('day', F.to_date('day')) \
        .withColumn('year_month', F.date_format('day', 'yyyy-MM')) \
        .select('day', 'year_month', 'origin_country', 'destination_country', 'origin_icao', 'destination_icao')
    # map country abbreviation to country name
    df_flight_mapped = df_flight_mapped.join(countries, df_flight_mapped['origin_country'] == countries['country_code'], 'left') \
        .drop('country_code') \
        .withColumnRenamed('origin_country', 'origin_country_code').withColumnRenamed('country', 'origin_country')
    df_flight_mapped = df_flight_mapped.join(countries, df_flight_mapped['destination_country'] == countries['country_code'], 'left') \
        .drop('country_code') \
        .withColumnRenamed('destination_country', 'destination_country_code').withColumnRenamed('country', 'destination_country')
    
    # drop rows with missing country or date
    df_flight_mapped = df_flight_mapped.filter(F.col('origin_country').isNotNull() & F.col('destination_country').isNotNull() & F.col('day').isNotNull())

    return df_flight_mapped

df_flight_mapped = map_flight_data(df_flight, airports, countries)

df_flight_mapped.show(5)

+----------+----------+-------------------+------------------------+-----------+----------------+--------------+-------------------+
|       day|year_month|origin_country_code|destination_country_code|origin_icao|destination_icao|origin_country|destination_country|
+----------+----------+-------------------+------------------------+-----------+----------------+--------------+-------------------+
|2018-12-31|   2018-12|                 AU|                      FR|       YMML|            LFPG|     Australia|             France|
|2018-12-31|   2018-12|                 AU|                      ES|       YMML|            LEBL|     Australia|              Spain|
|2018-12-31|   2018-12|                 AU|                      DE|       YSSY|            EDDF|     Australia|            Germany|
|2018-12-31|   2018-12|                 ES|                      ES|       LEMD|            LEMD|         Spain|              Spain|
|2018-12-31|   2018-12|                 AU|                      FR| 

In [57]:
def process_flight_data(df: DataFrame, direction: str = 'origin', granular: str = 'day', country: list[str] = None) -> DataFrame:
    """
    Process the daily or monthly data
    Args:
        df: DataFrame, the flight data
        direction: str, the direction of the flight
        granular: str, the granularity of the data
        country: list[str], the country to filter the data
    Returns:
        df_agg: DataFrame, the aggregated data
    """
    assert direction in ['origin', 'destination']
    assert granular in ['day', 'year_month']
    opposite_direction = 'origin' if direction == 'destination' else 'destination'
    if country:
        df = df.filter(F.col(f'{direction}_country').isin(country))
    # exclude domestic flight
    df = df.filter(F.col('origin_country_code') != F.col('destination_country_code'))
    # aggrgate by country and month or day
    df_agg = df.groupBy(f'{granular}', 'origin_country', 'origin_country_code', 'destination_country', 'destination_country_code').agg(
        F.count('*').alias(f'flight_count')
    ).sort(f'{direction}_country_code', f'{opposite_direction}_country_code', f'{granular}')
    return df_agg

df_daily = process_flight_data(df_flight_mapped, 'origin', 'day')
df_daily.show(5)

df_monthly = process_flight_data(df_flight_mapped, 'origin', 'year_month')
df_monthly.show(5)




+----------+--------------------+-------------------+-------------------+------------------------+------------+
|       day|      origin_country|origin_country_code|destination_country|destination_country_code|flight_count|
+----------+--------------------+-------------------+-------------------+------------------------+------------+
|2018-12-31|United Arab Emirates|                 AE|            Austria|                      AT|           2|
|2019-01-01|United Arab Emirates|                 AE|            Austria|                      AT|           2|
|2019-01-02|United Arab Emirates|                 AE|            Austria|                      AT|           3|
|2019-01-03|United Arab Emirates|                 AE|            Austria|                      AT|           2|
|2019-01-04|United Arab Emirates|                 AE|            Austria|                      AT|           2|
+----------+--------------------+-------------------+-------------------+------------------------+------

+----------+--------------------+-------------------+-------------------+------------------------+------------+
|year_month|      origin_country|origin_country_code|destination_country|destination_country_code|flight_count|
+----------+--------------------+-------------------+-------------------+------------------------+------------+
|   2018-12|United Arab Emirates|                 AE|            Austria|                      AT|           2|
|   2019-01|United Arab Emirates|                 AE|            Austria|                      AT|          70|
|   2018-12|United Arab Emirates|                 AE|          Australia|                      AU|          13|
|   2019-01|United Arab Emirates|                 AE|          Australia|                      AU|         474|
|   2019-01|United Arab Emirates|                 AE|              Aruba|                      AW|           1|
+----------+--------------------+-------------------+-------------------+------------------------+------